# Exploring Dynamic Machine Learning Methods for FX Portfolio Risk Management

In [68]:
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import *
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import GridSearchCV
 
sys.path.append('..')
# from .. Trading import *
sys.path
from Trading import *

# Decision Trees

In [70]:
# hola = Batch(start='1999-01-01', end='2000-01-01')
# hola.px_data


Decision_trees.ipynb __init__.py


# Random Forests